In [12]:
# Create server parameters for stdio connection
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

from langchain_mcp_adapters.tools import load_mcp_tools
from langchain.agents import create_agent

from IPython.display import Markdown
import os

# Or load from .env file
from dotenv import load_dotenv
load_dotenv()

from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain.agents import create_agent
from langchain.agents.middleware import (
    SummarizationMiddleware,
    TodoListMiddleware,
    ShellToolMiddleware,
    ToolRetryMiddleware,
)
from langchain.agents.middleware import wrap_tool_call
from langchain.messages import ToolMessage
from utils.md_to_pdf import markdown_folder_to_pdf

### One Server MCP

In [2]:
server_params = StdioServerParameters(
    command="python",
    # Make sure to update to the full absolute path to your math_server.py file
    args=["/home/nitish/Documents/github/Langchain/tools/mcp_search_server.py"],
)

async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        # Initialize the connection
        await session.initialize()

        # Get tools
        tools = await load_mcp_tools(session)

        # Create and run the agent
        agent = create_agent("openai:gpt-5-mini", tools)
        agent_response = await agent.ainvoke({"messages": "what's the latest news on India US relations?"})

Markdown(agent_response['messages'][-1].content)

Here are the main, recent developments (late November 2025) in India–US relations:

- Arms sale approved (20 Nov 2025)
  - The US cleared a roughly $93 million package including FGM‑148 Javelin anti‑tank missiles and M982A1 Excalibur precision artillery rounds. The State Department/DSCA said the sales strengthen the strategic relationship and India can absorb the equipment. (BBC, multiple Indian outlets)

- Trade negotiations advancing; aim to seal first tranche by year‑end
  - India and the US have been holding regular virtual trade talks. India’s commerce secretary Rajesh Agarwal said most issues are resolved and the first tranche of a Bilateral Trade Agreement (and a parallel tariff framework) is expected to be locked before the end of 2025. Negotiations remain two‑track: a broader BTA and a focused deal to address reciprocal tariffs. (Economic Times, NDTV, Reuters coverage)

- Tariff/tension background
  - Earlier in 2025 the US imposed steep “reciprocal” and Russia‑related penalties (together reaching about 50% on many Indian exports). Those tariffs are a key item in the current talks; India seeks duty concessions for labour‑intensive sectors (textiles, gems, leather, seafood, etc.) while the US seeks concessions in certain industrial, auto (EV), and agricultural items.

- Strategic/defence ties and broader context
  - Defence cooperation has been growing (including recent framework agreements to expand cooperation). India continues to diversify away from Russian suppliers toward more US procurement, even as Russia remains a major source. The arms sale and talks reflect a strengthening but pragmatic strategic partnership amid trade frictions.

- Other signals
  - India’s regional clout is rising (Asia Power Index 2025 ranks India high), and both governments appear motivated to resolve economic disputes while deepening defence and strategic cooperation.

If you want, I can:
- Pull and link the latest full news articles (BBC, Economic Times, NDTV, Business Standard) for reading;
- Summarize the trade deal positions in more detail (what each side wants and likely sticking points);
- Create a short timeline of key events in 2025 (tariff moves, meetings, agreements, arms sales). Which would you prefer?

### Multi-Server MCP

In [ ]:


@wrap_tool_call
def handle_tool_errors(request, handler):
    """Handle tool execution errors with custom messages."""
    try:
        return handler(request)
    except Exception as e:
        # Return a custom error message to the model
        return ToolMessage(
            content=f"Tool error: Please check your input and try again. ({str(e)})",
            tool_call_id=request.tool_call["id"]
        )
from pathlib import Path

client = MultiServerMCPClient(
    {
        "filesystem": {
            "command": "python",
            "args": ["/home/nitish/Documents/github/Langchain/tools/mcp_filesystem.py"],
            "transport": "stdio",
        },
        "search": {
            "command": "python",
            "args": ["/home/nitish/Documents/github/Langchain/tools/mcp_search_server.py"],
            "transport": "stdio",
        },
    }
)
tools = await client.get_tools()

# Configure middleware for the agent
middleware = [
    SummarizationMiddleware("groq:openai/gpt-oss-20b", max_tokens_before_summary = 40000),
    TodoListMiddleware(),
    ShellToolMiddleware(
        workspace_root="/home/nitish/Documents/github/Langchain",
        startup_commands=None,
        shutdown_commands=None,
    ),
    ToolRetryMiddleware(
        max_retries=3,  # Maximum number of retry attempts
        backoff_factor=1.0,  # Exponential backoff factor between retries
    ),
]

# Create and run the agent using Google AI (Gemini) - free for personal use
#agent = create_agent("google_genai:models/gemini-flash-latest", tools, middleware=middleware)
#agent = create_agent("openai:gpt-5-mini", tools, middleware=middleware)
agent = create_agent("anthropic:claude-haiku-4-5-20251001", tools, middleware=middleware)

#agent = create_agent("groq:openai/gpt-oss-120b", tools, middleware=middleware)
#agent = create_agent("ollama:hf.co/unsloth/Qwen3-30B-A3B-Thinking-2507-GGUF:Q4_K_M", tools, middleware=middleware)

response = await agent.ainvoke({"messages": """Define Autoencoders - use wiki. But mention in details. 
                                Then, prepare a 3 chapter course on them. Chapter 1 describes the basics and the mechanism. Feel free to use mermaid diagrams if necessary. 
                                The next chapter shows applications and the appropriate type of autoencoder. 
                                The last chapter shows tensorflow code for 3 kinds of autoencoders.
                                write them in separate markdown files - in Autoencoders_Course folder."""})
Markdown(response['messages'][-1].content)

In [3]:
response

{'messages': [HumanMessage(content='Here is a summary of the conversation to date:\n\nPrevious conversation was too long to summarize.', additional_kwargs={}, response_metadata={}, id='76d8076a-fe03-4725-b6cf-18dba8f71763'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 780, 'prompt_tokens': 10095, 'total_tokens': 10875, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 576, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 4736}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-ChEz9itR7wbsSwLskIHtr5WmIoWy3', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--a8e1a1fb-358c-4674-8047-f3998fbfb574-0', tool_calls=[{'name': 'write_todos', 'args': {'todos': [{'content': "Fetch Wikipedia article on 'Autoencoder' and extract d

### Naive General Purpose Agent

In [6]:
from pathlib import Path

client = MultiServerMCPClient(
    {
        "filesystem": {
            "command": "python",
            "args": ["/home/nitish/Documents/github/Langchain/tools/mcp_filesystem.py"],
            "transport": "stdio",
        },
        "search": {
            "command": "python",
            "args": ["/home/nitish/Documents/github/Langchain/tools/mcp_search_server.py"],
            "transport": "stdio",
        },

        "subagents": {
            "command": "python",
            "args": ["/home/nitish/Documents/github/Langchain/tools/mcp_subagent.py"],
            "transport": "stdio",
        },
    }
)
tools = await client.get_tools()

# Configure middleware for the agent
middleware = [
    SummarizationMiddleware("groq:openai/gpt-oss-20b", max_tokens_before_summary = 40000),
    TodoListMiddleware(),
    ShellToolMiddleware(
        workspace_root="/home/nitish/Documents/github/Langchain",
        startup_commands=None,
        shutdown_commands=None,
    ),
    ToolRetryMiddleware(
        max_retries=3,  # Maximum number of retry attempts
        backoff_factor=1.0,  # Exponential backoff factor between retries
    ),
]

# Create and run the agent using Google AI (Gemini) - free for personal use
#agent = create_agent("google_genai:models/gemini-flash-latest", tools, middleware=middleware)
agent = create_agent("openai:gpt-5-mini", tools, middleware=middleware)
#agent = create_agent("anthropic:claude-haiku-4-5-20251001", tools, middleware=middleware)

#agent = create_agent("groq:openai/gpt-oss-120b", tools, middleware=middleware)
#agent = create_agent("ollama:hf.co/unsloth/Qwen3-30B-A3B-Thinking-2507-GGUF:Q4_K_M", tools, middleware=middleware)

response = await agent.ainvoke({"messages": """Create a new folder called SQL Agent - in that folder, create a project in python which builds a basic UI chatbot and uses langchain to communicate with LLM. Ping ollama on the system to check the model list and use gemma3:4b as the model. Think, Plan and execute. The UI must be simple htm,l css js """})
Markdown(response['messages'][-1].content)

Done — I created the project and scaffolding and checked the local Ollama model list.

What I created
- Project folder: SQL Agent
- Backend: FastAPI app at SQL Agent/app/main.py
  - Serves static files from SQL Agent/static
  - Endpoints:
    - GET / -> returns static/index.html
    - GET /api/models -> attempts to list models (falls back to configured model)
    - POST /api/chat -> sends the user's message to the configured Ollama model via LangChain and returns the LLM response
  - Configurable model via .env OLLAMA_MODEL (default: gemma3:4b)
- Frontend: simple HTML/CSS/JS in SQL Agent/static
  - index.html, styles.css, app.js
  - Basic chat UI and fetch-based communication with /api/chat
- Requirements: SQL Agent/requirements.txt
  - fastapi, uvicorn, langchain, ollama, httpx, python-dotenv
- README with quick run instructions
- .env with OLLAMA_MODEL=gemma3:4b

Ollama check
- I ran ollama list inside the environment. Output shows the header (NAME ID SIZE MODIFIED) but no models were listed. That means ollama is reachable but no models are currently pulled on the machine, or you need to run ollama pull gemma3:4b (or install model) locally.
- The service is configured to use gemma3:4b by default; if you have that model pulled it will be used.

How to run locally
1. Ensure Ollama is running locally and model gemma3:4b is available:
   - ollama list   # to see models
   - ollama pull gemma3:4b   # if needed
2. Create and activate Python venv:
   - python -m venv venv
   - source venv/bin/activate   (or venv\\Scripts\\activate on Windows)
3. Install dependencies:
   - pip install -r "SQL Agent/requirements.txt"
4. Start the FastAPI app:
   - cd "SQL Agent"
   - uvicorn app.main:app --reload
5. Open http://127.0.0.1:8000 in your browser and try chatting.

Notes and caveats
- The LangChain Ollama integration depends on your installed langchain and ollama client versions. The code uses Ollama from langchain.llms. If your versions differ, you may need to adapt initialization (e.g., supply client options, API host/port). The code falls back gracefully for /api/models.
- This is a minimal example intended as a starting point. For production use you should add error handling, rate limiting, sanitization, and authentication as needed.

If you want I can:
- Modify the backend to stream responses to the UI (for token-by-token updates).
- Add a small example showing SQL generation/parsing (since it's named SQL Agent).
- Add Dockerfile to containerize the app.
- Adjust code to a specific langchain/ollama client version you are using — tell me the version and I’ll adapt.

### Deep Agent General

In [ ]:
from pathlib import Path

client = MultiServerMCPClient(
    {
        "filesystem": {
            "command": "python",
            "args": ["/home/nitish/Documents/github/Langchain/tools/mcp_filesystem.py"],
            "transport": "stdio",
        },
        "search": {
            "command": "python",
            "args": ["/home/nitish/Documents/github/Langchain/tools/mcp_search_server.py"],
            "transport": "stdio",
        },

        "subagents": {
            "command": "python",
            "args": ["/home/nitish/Documents/github/Langchain/tools/mcp_subagent.py"],
            "transport": "stdio",
        },
    }
)
tools = await client.get_tools()

# Configure middleware for the agent
middleware = [
    SummarizationMiddleware("groq:openai/gpt-oss-20b", max_tokens_before_summary = 40000),
    TodoListMiddleware(),
    ShellToolMiddleware(
        workspace_root="/home/nitish/Documents/github/Langchain",
        startup_commands=None,
        shutdown_commands=None,
    ),
    ToolRetryMiddleware(
        max_retries=3,  # Maximum number of retry attempts
        backoff_factor=1.0,  # Exponential backoff factor between retries
    ),
]

# Create and run the agent using Google AI (Gemini) - free for personal use
#agent = create_agent("google_genai:models/gemini-flash-latest", tools, middleware=middleware)
agent = create_agent("openai:gpt-5-mini", tools, middleware=middleware)
#agent = create_agent("anthropic:claude-haiku-4-5-20251001", tools, middleware=middleware)

#agent = create_agent("groq:openai/gpt-oss-120b", tools, middleware=middleware)
#agent = create_agent("ollama:hf.co/unsloth/Qwen3-30B-A3B-Thinking-2507-GGUF:Q4_K_M", tools, middleware=middleware)

response = await agent.ainvoke({"messages": """Create a new folder called SQL Agent - in that folder, create a project in python which builds a basic UI chatbot and uses langchain to communicate with LLM. Ping ollama on the system to check the model list and use gemma3:4b as the model. Think, Plan and execute. The UI must be simple htm,l css js """})
Markdown(response['messages'][-1].content)

Advanced middleware configuration with MCP loaded successfully
Configured servers: filesystem, search
Available tools: 10


### Deep Agent with MCP + Planning + Search + Filesystem
Replacement for the Claude-oriented agent using LangChain's `create_deep_agent` with the same MCP backends.


In [11]:
from deepagents import create_deep_agent
deep_system_prompt = """You are a planning-first MCP agent replacing the earlier Claude setup.
- Start every task with a concise, ordered plan before executing tools.
- Prefer MCP filesystem tools for reads/writes; keep changes inside /home/nitish/Documents/github/Langchain.
- Use the search server (web_search/open_url) for current or unknown information and cite URLs.
- Call sub-agents when specialized help is useful; summarize their output.
- Keep shell commands minimal and safe.
- End with a short recap of actions taken and files touched.
"""

client = MultiServerMCPClient(
    {
        "filesystem": {
            "command": "python",
            "args": ["/home/nitish/Documents/github/Langchain/tools/mcp_filesystem.py"],
            "transport": "stdio",
        },
        "search": {
            "command": "python",
            "args": ["/home/nitish/Documents/github/Langchain/tools/mcp_search_server.py"],
            "transport": "stdio",
        },
        "subagents": {
            "command": "python",
            "args": ["/home/nitish/Documents/github/Langchain/tools/mcp_subagent.py"],
            "transport": "stdio",
        },
    }
)
tools = await client.get_tools()

middleware = [
    SummarizationMiddleware("groq:openai/gpt-oss-20b", max_tokens_before_summary=40000),
    TodoListMiddleware(),
    ShellToolMiddleware(
        workspace_root="/home/nitish/Documents/github/Langchain",
        startup_commands=None,
        shutdown_commands=None,
    ),
    ToolRetryMiddleware(max_retries=3, backoff_factor=1.0),
]
middleware = [
]
deep_agent = create_deep_agent(
    model="openai:gpt-5-mini",
    tools=tools,
    system_prompt=deep_system_prompt,
    middleware=middleware,
)

deep_response = await deep_agent.ainvoke({
    "messages": """Create a new folder called SQL Agent 2 - in that folder, create a project in python which builds a basic UI chatbot and uses langchain to communicate with LLM. Ping ollama on the system to check the model list and use gemma3:4b as the model. Think, Plan and execute. The UI must be simple htm,l css js """
})
Markdown(deep_response["messages"][-1].content)


Plan:
1. Create a project folder "SQL Agent 2" under /home/nitish/Documents/github/Langchain and scaffold files (FastAPI backend, static HTML/CSS/JS, templates, requirements).
2. Implement FastAPI app that uses LangChain's Ollama LLM set to model gemma3:4b and add a /chat endpoint.
3. Add a /models endpoint that pings the local Ollama HTTP API (to verify installed models).
4. Add a simple static HTML/CSS/JS chat UI that POSTs to /chat.
5. Run ollama list to confirm gemma3:4b is available and save the output.
6. Update README and mark todos complete. Report files created and next steps.

What I created and changed
- Created project directory: /home/nitish/Documents/github/Langchain/SQL Agent 2
- Files added:
  - app.py — FastAPI backend using langchain.llms.Ollama and a /chat POST route and /models GET route that calls the local Ollama HTTP API.
  - requirements.txt — listing fastapi, uvicorn, langchain, ollama, python-dotenv, requests.
  - .env — default OLLAMA_BASE_URL and OLLAMA_MODEL (gemma3:4b).
  - README.md — run instructions and note about checking models.
  - ollama_models.txt — saved output from running ollama list in the project directory.
  - templates/index.html — simple chat UI.
  - static/styles.css — CSS for the UI.
  - static/app.js — JavaScript to POST messages to /chat and render replies.

Key implementation notes
- Backend (app.py)
  - Loads .env with python-dotenv.
  - Configures MODEL_NAME from OLLAMA_MODEL env var (defaults to gemma3:4b).
  - Uses LangChain's Ollama wrapper: Ollama(model=MODEL_NAME, base_url=OLLAMA_BASE_URL).
  - POST /chat accepts JSON {message: "..."} and returns {reply: "..."}.
  - GET /models hits the local Ollama HTTP API at {OLLAMA_BASE_URL}/models to return the list of available models.
  - Basic error handling returns JSON error messages.
- Frontend
  - Minimal single-page HTML that posts to /chat and appends messages to the chat box (vanilla JS + CSS).
  - No build step — serve static files via FastAPI static mount.
- ollama model check
  - I ran ollama list in the project directory; the output (saved to ollama_models.txt) includes gemma3:4b (ID a2af6cc3eb7f) confirming it's present locally.

Exact files created (paths relative to /home/nitish/Documents/github/Langchain)
- SQL Agent 2/
  - app.py
  - requirements.txt
  - .env
  - README.md
  - ollama_models.txt
  - templates/
    - index.html
  - static/
    - styles.css
    - app.js

How to run locally
1. cd /home/nitish/Documents/github/Langchain/SQL Agent 2
2. python -m pip install -r requirements.txt
   - Note: depending on your environment, you may need to install or configure the Ollama client or LangChain extras.
3. Ensure the Ollama daemon is running locally (default http://localhost:11434) and that gemma3:4b is already pulled/installed (you can run: ollama list).
4. Start the API:
   - uvicorn app:app --reload
5. Open http://localhost:8000 in your browser to use the chat UI.
6. You can also GET http://localhost:8000/models to see the JSON returned from the Ollama daemon.

Safety & limitations
- This is a simple demo that sends user text directly to the LLM. Do not expose the service publicly without authentication or rate limiting.
- The LangChain Ollama wrapper uses the Ollama client libraries; ensure version compatibility with your installed packages.
- If Ollama is not running at the default port, set OLLAMA_BASE_URL in .env.

Files touched/created (recap)
- Created: SQL Agent 2/app.py
- Created: SQL Agent 2/requirements.txt
- Created: SQL Agent 2/.env
- Created: SQL Agent 2/README.md (edited after creation)
- Created: SQL Agent 2/ollama_models.txt (output of ollama list)
- Created: SQL Agent 2/templates/index.html
- Created: SQL Agent 2/static/styles.css
- Created: SQL Agent 2/static/app.js

Todos: completed
- Project scaffolded, backend implemented, frontend created, ollama models checked and saved, README updated.

If you want I can:
- Add CORS middleware for remote clients, or
- Add message streaming support (if Ollama/LangChain client supports streaming), or
- Add basic authentication, or
- Create a Dockerfile to containerize the app.

Would you like me to add any of those, or test a live request through the LLM (I can simulate a request but I won't call your local Ollama instance without your confirmation)?

In [ ]:
# Interactive Conversation Loop - Run this cell to continue the conversation
# The checkpointer maintains conversation history across invocations

def continue_conversation():
    """Get user input and continue the conversation with memory."""
    user_input = input("\n💬 Your message (or 'quit' to exit): ")
    return user_input.strip()

# Continue conversation loop
while True:
    user_message = continue_conversation()
    
    if user_message.lower() in ['quit', 'exit', 'q', '']:
        print("\n👋 Conversation ended. Thread history preserved.")
        print(f"   Thread ID: {thread_id}")
        break
    
    print(f"\n📝 You: {user_message}")
    print("⏳ Agent is thinking...")
    
    # Continue conversation with same thread_id (memory persists)
    deep_response = await deep_agent.ainvoke(
        {"messages": user_message},
        config=config,  # Same config = same conversation thread
    )
    
    agent_reply = deep_response["messages"][-1].content
    print(f"\n🤖 Agent:")
    display(Markdown(agent_reply))

### Resume a Previous Conversation
Run the cell below to resume a conversation using a saved thread_id. The checkpointer will restore the full conversation history.

In [ ]:
# Resume a previous conversation by providing the thread_id
# Uncomment and set your thread_id to resume

# saved_thread_id = "your-previous-thread-id-here"
# config = {"configurable": {"thread_id": saved_thread_id}}

# Then run the conversation loop cell above to continue that conversation

## Plus User Intervention

In [ ]:
from deepagents import create_deep_agent
deep_system_prompt = """You are a planning-first MCP agent replacing the earlier Claude setup.
- Start every task with a concise, ordered plan before executing tools.
- Prefer MCP filesystem tools for reads/writes; keep changes inside /home/nitish/Documents/github/Langchain.
- Use the search server (web_search/open_url) for current or unknown information and cite URLs.
- Call sub-agents when specialized help is useful; summarize their output.
- Keep shell commands minimal and safe.
- End with a short recap of actions taken and files touched.
"""

client = MultiServerMCPClient(
    {
        "filesystem": {
            "command": "python",
            "args": ["/home/nitish/Documents/github/Langchain/tools/mcp_filesystem.py"],
            "transport": "stdio",
        },
        "search": {
            "command": "python",
            "args": ["/home/nitish/Documents/github/Langchain/tools/mcp_search_server.py"],
            "transport": "stdio",
        },
        "subagents": {
            "command": "python",
            "args": ["/home/nitish/Documents/github/Langchain/tools/mcp_subagent.py"],
            "transport": "stdio",
        },
    }
)
tools = await client.get_tools()

middleware = [
    SummarizationMiddleware("groq:openai/gpt-oss-20b", max_tokens_before_summary=40000),
    TodoListMiddleware(),
    ShellToolMiddleware(
        workspace_root="/home/nitish/Documents/github/Langchain",
        startup_commands=None,
        shutdown_commands=None,
    ),
    ToolRetryMiddleware(max_retries=3, backoff_factor=1.0),
]
middleware = [
]
deep_agent = create_deep_agent(
    model="openai:gpt-5-mini",
    tools=tools,
    system_prompt=deep_system_prompt,
    middleware=middleware,
)

deep_response = await deep_agent.ainvoke({
    "messages": """Create a new folder called SQL Agent 2 - in that folder, create a project in python which builds a basic UI chatbot and uses langchain to communicate with LLM. Ping ollama on the system to check the model list and use gemma3:4b as the model. Think, Plan and execute. The UI must be simple htm,l css js """
})
Markdown(deep_response["messages"][-1].content)
